In [ ]:
#!pip install tensorflow
import pandas as pd
import tensorflow as tf
import numpy as np
import os
import scipy.io
import h5py
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
import pickle
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from IPython.display import clear_output
from scipy.signal import savgol_filter
import gc

In [ ]:
def load_data(file_name,pickle_name,bn):
  f = h5py.File(file_name,"r")
  print(list(f.keys()))
  batch=f.get("batch")
  num_cells = batch['summary'].shape[0]
  bat_dict = {}
  for i in range(num_cells):
    cl = f[batch['cycle_life'][i,0]][()] #ici on a remplacé ".value" par "[()]", et dans toutes les lignes suivantes
    policy = f[batch['policy_readable'][i,0]][()].tobytes()[::2].decode()
    summary_IR = np.hstack(f[batch['summary'][i,0]]['IR'][0,:].tolist())
    summary_QC = np.hstack(f[batch['summary'][i,0]]['QCharge'][0,:].tolist())
    summary_QD = np.hstack(f[batch['summary'][i,0]]['QDischarge'][0,:].tolist())
    summary_TA = np.hstack(f[batch['summary'][i,0]]['Tavg'][0,:].tolist())
    summary_TM = np.hstack(f[batch['summary'][i,0]]['Tmin'][0,:].tolist())
    summary_TX = np.hstack(f[batch['summary'][i,0]]['Tmax'][0,:].tolist())
    summary_CT = np.hstack(f[batch['summary'][i,0]]['chargetime'][0,:].tolist())
    summary_CY = np.hstack(f[batch['summary'][i,0]]['cycle'][0,:].tolist())
    summary = ({'IR': summary_IR, #création d'un 1er dict avec les données "résumées", càd 1 donnée par cycle
               'QC': summary_QC, 
               'QD': summary_QD, 
               'Tavg': summary_TA, 
               'Tmin': summary_TM, 
               'Tmax': summary_TX, 
               'chargetime': summary_CT,
               'cycle': summary_CY}) 
    cycles = f[batch['cycles'][i,0]]
    cycle_dict = {}
    for j in range(cycles['I'].shape[0]):
        I = np.hstack((f[cycles['I'][j,0]][()]))
        Qc = np.hstack((f[cycles['Qc'][j,0]][()]))
        Qd = np.hstack((f[cycles['Qd'][j,0]][()]))
        Qdlin = np.hstack((f[cycles['Qdlin'][j,0]][()]))
        T = np.hstack((f[cycles['T'][j,0]][()]))
        Tdlin = np.hstack((f[cycles['Tdlin'][j,0]][()]))
        V = np.hstack((f[cycles['V'][j,0]][()]))
        dQdV = np.hstack((f[cycles['discharge_dQdV'][j,0]][()]))
        t = np.hstack((f[cycles['t'][j,0]][()]))
        cd = ({'I': I,  
               'Qc': Qc, 
               'Qd': Qd, 
               'Qdlin': Qdlin, 
               'T': T, 
               'Tdlin': Tdlin, 
               'V':V, 
               'dQdV': dQdV, 
               't':t})
        cycle_dict[str(j)] = cd
        
    cell_dict = ({'cycle_life': cl, 
                 'charge_policy':policy, 
                  'summary': summary, 
                  'cycles': cycle_dict})
    key = "b"+bn+"c" + str(i)
    bat_dict[key]=   cell_dict


  df=pd.DataFrame(bat_dict)

  with open(pickle_name,'wb') as fp:
        pickle.dump(bat_dict,fp) 
  return df

In [ ]:
path_to_file=""

In [ ]:
b1=load_data(path_to_file+"2017-05-12_batchdata_updated_struct_errorcorrect.mat","batch1.pkl","1")
b2=load_data(path_to_file+"2017-06-30_batchdata_updated_struct_errorcorrect.mat","batch2.pkl","2")
b3=load_data(path_to_file+"2018-04-12_batchdata_updated_struct_errorcorrect.mat","batch3.pkl","3")